In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df=sb.load_dataset("tips")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
df.describe

<bound method NDFrame.describe of      total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null category
smoker        244 non-null category
day           244 non-null category
time          244 non-null category
size          244 non-null int64
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
df1=df.groupby(['day','size']).sum()
df1

total_bill     tip
day  size                    
Thur 1          10.07    1.83
     2         727.53  117.24
     3          76.64   10.77
     4         149.75   21.09
     5          41.19    5.00
     6          91.15   15.90
Fri  1           8.58    1.92
     2         261.15   42.31
     3          15.98    3.00
     4          40.17    4.73
     5            NaN     NaN
     6            NaN     NaN
Sat  1          10.32    2.00
     2         892.37  133.43
     3         459.17   68.36
     4         388.39   53.61
     5          28.15    3.00
     6            NaN     NaN
Sun  1            NaN     NaN
     2         684.84  109.86
     3         332.76   46.81
     4         480.39   73.58
     5          81.00   12.14
     6          48.17    5.00

In [7]:
df2=df1.dropna(axis=0)
df2

total_bill     tip
day  size                    
Thur 1          10.07    1.83
     2         727.53  117.24
     3          76.64   10.77
     4         149.75   21.09
     5          41.19    5.00
     6          91.15   15.90
Fri  1           8.58    1.92
     2         261.15   42.31
     3          15.98    3.00
     4          40.17    4.73
Sat  1          10.32    2.00
     2         892.37  133.43
     3         459.17   68.36
     4         388.39   53.61
     5          28.15    3.00
Sun  2         684.84  109.86
     3         332.76   46.81
     4         480.39   73.58
     5          81.00   12.14
     6          48.17    5.00

In [8]:
df.replace({'sex':{'Male':0,'Female':1},'smoker':{'No':0,'Yes':1}},inplace=True)
df.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,0,0,Sat,Dinner,3
240,27.18,2.00,1,1,Sat,Dinner,2
241,22.67,2.00,0,1,Sat,Dinner,2
242,17.82,1.75,0,0,Sat,Dinner,2
243,18.78,3.00,1,0,Thur,Dinner,2


In [9]:
days=pd.get_dummies(df['day'],drop_first=True)
df=pd.concat([df,days],axis=1)
df.tail()

,total_bill,tip,sex,smoker,day,time,size,Fri,Sat,Sun
239,29.03,5.92,0,0,Sat,Dinner,3,0,1,0
240,27.18,2.00,1,1,Sat,Dinner,2,0,1,0
241,22.67,2.00,0,1,Sat,Dinner,2,0,1,0
242,17.82,1.75,0,0,Sat,Dinner,2,0,1,0
243,18.78,3.00,1,0,Thur,Dinner,2,0,0,0


In [10]:
times=pd.get_dummies(df['time'],drop_first=True)
df=pd.concat([df,times],axis=1)
df.head()

,total_bill,tip,sex,smoker,day,time,size,Fri,Sat,Sun,Dinner
0,16.99,1.01,1,0,Sun,Dinner,2,0,0,1,1
1,10.34,1.66,0,0,Sun,Dinner,3,0,0,1,1
2,21.01,3.50,0,0,Sun,Dinner,3,0,0,1,1
3,23.68,3.31,0,0,Sun,Dinner,2,0,0,1,1
4,24.59,3.61,1,0,Sun,Dinner,4,0,0,1,1


In [11]:
x=df[['total_bill','sex','smoker','size','Fri','Sat','Sun','Dinner']]
y=df[['tip']]

In [12]:
print(x,y)

     total_bill  sex  smoker  size  Fri  Sat  Sun  Dinner
0         16.99    1       0     2    0    0    1       1
1         10.34    0       0     3    0    0    1       1
2         21.01    0       0     3    0    0    1       1
3         23.68    0       0     2    0    0    1       1
4         24.59    1       0     4    0    0    1       1
..          ...  ...     ...   ...  ...  ...  ...     ...
239       29.03    0       0     3    0    1    0       1
240       27.18    1       1     2    0    1    0       1
241       22.67    0       1     2    0    1    0       1
242       17.82    0       0     2    0    1    0       1
243       18.78    1       0     2    0    0    0       1

[244 rows x 8 columns]       tip
0    1.01
1    1.66
2    3.50
3    3.31
4    3.61
..    ...
239  5.92
240  2.00
241  2.00
242  1.75
243  3.00

[244 rows x 1 columns]


In [13]:
Xtrain=df[['total_bill','tip','sex','smoker','Fri','Sat','Sun','Dinner']]
Ytrain=df[['size']]

Xtest=df[['total_bill','tip','sex','smoker','Fri','Sat','Sun','Dinner']]
Ytest=df[['size']]

In [14]:
from sklearn.linear_model import LogisticRegression as LR1

In [15]:
log_reg=LR1()

In [16]:
log_reg.fit(Xtrain,Ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
y_predict=log_reg.predict(Xtest)

In [18]:
print(y_predict)

[2 2 2 2 2 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 4 2 2 2 2 4 2 2 4 4 2 2 2 4 2 4 2 4 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2
 2 4 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 2
 2 2 2 2 2 2 2 4 4 2 2 2 2 2 2 2 2 2 2 4 2 2 4 2 2 2 2 2 2 2 2 4 4 2 4 2 4
 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2]


In [19]:
print(Ytest)

     size
0       2
1       3
2       3
3       2
4       4
..    ...
239     3
240     2
241     2
242     2
243     2

[244 rows x 1 columns]


In [20]:
if len(Ytest)==len(y_predict) : print("Length compatible")

Length compatible


In [21]:
l1=list(Ytest)
l2=list(y_predict)

In [22]:
count=0
for m in range(len(l1)):
        if l1[m] == l2[m]:
            count += 1
            
print('matches=',count)
print('accuracy=',count*100/len(l1))

matches= 0
accuracy= 0.0


In [23]:
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
print(confusion_matrix(Ytest, y_predict))

[[  0   4   0   0   0   0]
 [  0 154   0   2   0   0]
 [  0  30   0   8   0   0]
 [  0  20   0  17   0   0]
 [  0   3   0   2   0   0]
 [  0   2   0   2   0   0]]


In [25]:
print(classification_report(Ytest,y_predict))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.72      0.99      0.83       156
           3       0.00      0.00      0.00        38
           4       0.55      0.46      0.50        37
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         4

    accuracy                           0.70       244
   macro avg       0.21      0.24      0.22       244
weighted avg       0.55      0.70      0.61       244



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
